In [ ]:
from pathlib import Path

import happy_wheel
from happy_wheel import DATA_ROOT, NUM_INDIVIDUALS, SPECIES
from happy_wheel.config import TrainConfig, ModelConfig, LossConfig, OptimizerConfig, DatasetConfig, Config
import ykaggle_core as ycore

ycore.set_gpu(0)

### 学習率scheduler

In [ ]:
import math

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe"

scheduler = happy_wheel.lr_schedulers.WarmUpCosineAnnealing()
lr = [math.log(scheduler(i, 1e-4)) for i in range(200)]

px.line(lr)

### Config設定

In [ ]:
image_height = 256
image_width = 384
fold = 0
base_exp_name = "test"

train_config = TrainConfig(epochs=30)
model_config = ModelConfig(
    name="efficientnet-b0",
    kwargs=dict(
        include_top=False,
        input_shape=(image_height, image_width, 3),
    ),
    num_output_class=len(SPECIES),
    num_identity=NUM_INDIVIDUALS
)

train_dataset_config = DatasetConfig(
    input_dir=DATA_ROOT / "preprocessed" / "train_images",
    batch_size=32,
    width=image_width,
    height=image_height,
    shuffle=True,
    label_csv_path=DATA_ROOT / "preprocessed" / f"train_cv{fold}.csv",
)
val_dataset_config = DatasetConfig(
    input_dir=DATA_ROOT / "preprocessed" / "train_images",
    batch_size=64,
    width=image_width,
    height=image_height,
    shuffle=False,
    label_csv_path=DATA_ROOT / "preprocessed" / f"valid_cv{fold}.csv",
)
config = Config(
    exp_name=f"{base_exp_name}/cv{fold}",
    train=train_config,
    model=model_config,
    loss=LossConfig(name="categorical_crossentropy"),
    optimizer=OptimizerConfig(name="adam"),
    train_dataset=train_dataset_config,
    validation_dataset=val_dataset_config,
    test_dataset=val_dataset_config,
)

### build model

In [ ]:
model = happy_wheel.train.create_model(config)

In [ ]:
model.summary()

### Data loader

In [ ]:
train_ds, _ = happy_wheel.dataset.build_train_dataloader(config.train_dataset)

In [ ]:
for batch in train_ds:
    break

In [ ]:
model.train_on_batch(*batch)